In [5]:
# set up and dependencies
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.types import Date
from sqlalchemy.orm import Session
from sqlalchemy import func
import pandas as pd 
from pprint import pprint

%load_ext nb_black

<IPython.core.display.Javascript object>

In [6]:
# set up Base

Base = declarative_base()

<IPython.core.display.Javascript object>

In [7]:
#create class for measurement table

class Measurement(Base):
    __tablename__ = "measurement"
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)

<IPython.core.display.Javascript object>

In [8]:
#create class for station table

class Station(Base):
    __tablename__ = "station"
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation =  Column(Float)

<IPython.core.display.Javascript object>

In [9]:
# create engine and session to link to the database

engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()
session = Session(bind=engine)

<IPython.core.display.Javascript object>

In [11]:
# look at first 5 of measurement table

measurement_rows = session.query(Measurement).limit(5)

for row in measurement_rows:
    pprint(row.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0be3a181d0>,
 'date': datetime.date(2010, 1, 1),
 'id': 1,
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65.0}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0be3a18b50>,
 'date': datetime.date(2010, 1, 2),
 'id': 2,
 'prcp': 0.0,
 'station': 'USC00519397',
 'tobs': 63.0}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0be3a18990>,
 'date': datetime.date(2010, 1, 3),
 'id': 3,
 'prcp': 0.0,
 'station': 'USC00519397',
 'tobs': 74.0}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0be3a70190>,
 'date': datetime.date(2010, 1, 4),
 'id': 4,
 'prcp': 0.0,
 'station': 'USC00519397',
 'tobs': 76.0}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0be3a70090>,
 'date': datetime.date(2010, 1, 6),
 'id': 5,
 'prcp': None,
 'station': 'USC00519397',
 'tobs': 73.0}


<IPython.core.display.Javascript object>

In [12]:
#check the max date in data
max_date = session.query(func.max(Measurement.date)).first()[0]

print(max_date)

2017-08-23


<IPython.core.display.Javascript object>

In [16]:
# get the last 12 months of precipitation data

recent_prcp = session.query(Measurement.date, func.round(func.sum(Measurement.prcp), 2)).filter(Measurement.date > '2016-08-23').filter(Measurement.date <= '2017-08-23').group_by(Measurement.date).order_by(Measurement.date).all()
        
pprint(recent_prcp)

[(datetime.date(2016, 8, 24), 9.33),
 (datetime.date(2016, 8, 25), 0.54),
 (datetime.date(2016, 8, 26), 0.1),
 (datetime.date(2016, 8, 27), 0.32),
 (datetime.date(2016, 8, 28), 3.1),
 (datetime.date(2016, 8, 29), 1.46),
 (datetime.date(2016, 8, 30), 0.07),
 (datetime.date(2016, 8, 31), 3.18),
 (datetime.date(2016, 9, 1), 0.03),
 (datetime.date(2016, 9, 2), 0.25),
 (datetime.date(2016, 9, 3), 1.27),
 (datetime.date(2016, 9, 4), 1.38),
 (datetime.date(2016, 9, 5), 0.34),
 (datetime.date(2016, 9, 6), 1.23),
 (datetime.date(2016, 9, 7), 2.0),
 (datetime.date(2016, 9, 8), 0.46),
 (datetime.date(2016, 9, 9), 1.07),
 (datetime.date(2016, 9, 10), 1.4),
 (datetime.date(2016, 9, 11), 1.25),
 (datetime.date(2016, 9, 12), 1.54),
 (datetime.date(2016, 9, 13), 2.71),
 (datetime.date(2016, 9, 14), 14.28),
 (datetime.date(2016, 9, 15), 4.96),
 (datetime.date(2016, 9, 16), 0.82),
 (datetime.date(2016, 9, 17), 0.68),
 (datetime.date(2016, 9, 18), 0.53),
 (datetime.date(2016, 9, 19), 0.32),
 (datetime.da

<IPython.core.display.Javascript object>